In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import cv2
import numpy as np
import os
from tensorflow.keras.utils import to_categorical
import json
from collections import Counter
import shutil
from skimage.feature import hog
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
from sklearn import svm
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Conv2D, MaxPooling2D, Flatten
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np
from sklearn.metrics import confusion_matrix
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Dropout
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.regularizers import l2
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
import warnings
from sklearn.exceptions import ConvergenceWarning
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from skimage.color import rgb2gray
import os
import joblib


In [3]:
GOOGLE_DRIVE_PATH_AFTER_MYDRIVE = '/content/drive/MyDrive/Colab Notebooks/Shreenika_CV_CW_Folder'
GOOGLE_DRIVE_PATH = os.path.join('drive', 'My Drive', GOOGLE_DRIVE_PATH_AFTER_MYDRIVE)
print(os.listdir(GOOGLE_DRIVE_PATH))

['CW_Dataset', 'Code', 'Models', 'Video', 'Copy of CNN_model.pkl', 'Copy of test_functions.ipynb']


In [4]:
"""The data is loaded from the train directory and into train_images and train_labels
The function iterates through the files and resizes each image to 128x128 pixels and
normalizes them to pixel values ranging [0,1]. Finally it retrieves the corresponding and
converts them to integer and applies on-hot encoding and appends both images and labels to separate lists.
"""

train_directory = os.path.join(GOOGLE_DRIVE_PATH,'CW_Dataset','train')

def load_images(images_directory, labels_directory):
    images = []
    labels = []

    for file in sorted(os.listdir(images_directory)):
        if file.endswith(".jpeg"):
            img_path = os.path.join(images_directory, file)
            img = cv2.imread(img_path)
            img = cv2.resize(img, (128, 128))
            img = img / 255.0
            images.append(img)

            label_file_name = file.replace(".jpeg", ".txt")
            label_file_path = os.path.join(labels_directory, label_file_name)

            with open(label_file_path, 'r') as label_file:
                label = int(label_file.read().strip())
                labels.append(label)

    return np.array(images), to_categorical(labels, num_classes=3)

train_images_directory = os.path.join(GOOGLE_DRIVE_PATH,'CW_Dataset','train','images')
train_labels_directory = os.path.join(GOOGLE_DRIVE_PATH,'CW_Dataset','train','labels')

train_images, train_labels = load_images(train_images_directory, train_labels_directory)


In [5]:
""" This method check each image if its in grayscale or color. if image is in color it
converts it into grayscale. it then computes HOG features using hog function with parameters
such as orientation bins, size of cells in pixels and number of cells per block.
"""
def extracting_hog_features(images):
    hog_features = []
    for img in images:
        if len(img.shape) == 3:
            gray_img = rgb2gray(img)
        else:
            gray_img = img

        if gray_img.size == 0:
            continue

        fd, hog_image = hog(gray_img, orientations=8, pixels_per_cell=(16, 16),
                            cells_per_block=(1, 1), visualize=True)
        hog_features.append(fd)
    return np.array(hog_features)


In [6]:
"""
this code is the preprocessing steps for face mask detection where the training data is
split into train and validation sets. And then hog features are extracted from the images to be
used in algorithms below
"""
X_train, X_val, y_train, y_val = train_test_split(train_images, train_labels, test_size=0.2, random_state=42)

print("Training Images Shape:", X_train.shape)
print("Validation Images Shape:", X_val.shape)

X_train_features = extracting_hog_features(X_train)

X_val_features = extracting_hog_features(X_val)

print("Train Features Shape:", X_train_features.shape)
print("Train Labels Shape:", y_train.shape)
print("Validation Features Shape:", X_val_features.shape)
print("Validation Labels Shape:", y_val.shape)

Training Images Shape: (1915, 128, 128, 3)
Validation Images Shape: (479, 128, 128, 3)
Train Features Shape: (1915, 512)
Train Labels Shape: (1915, 3)
Validation Features Shape: (479, 512)
Validation Labels Shape: (479, 3)


MLP+hog with Hyper parameter optimization

In [9]:
"""
This code performs grid search over various hyperparameter of Multilayer perceptron classifier.
the best hyperparameters are identified using cross-validation and new MLP classifier
instance is initiated using the best parameters and then its trained on the extracted
HOG features. Model is evaluated on both training and validation sets.
"""

warnings.filterwarnings("ignore", category=ConvergenceWarning)

y_train_indices = np.argmax(y_train, axis=1)
y_val_indices = np.argmax(y_val, axis=1)

param_grid = {
    'hidden_layer_sizes': [(64,), (128,), (256,), (64, 64), (128, 128), (256, 256)],
    'activation': ['relu', 'tanh'],
    'solver': ['adam'],
    'alpha': [0.0001, 0.001, 0.01],
    'learning_rate': ['constant', 'adaptive']
}

mlp_classifier = MLPClassifier(max_iter=5, random_state=42)

grid_search = GridSearchCV(mlp_classifier, param_grid, cv=3, scoring='accuracy')
grid_search.fit(X_train_features, y_train_indices)

best_params = grid_search.best_params_

mlp_classifier = MLPClassifier(hidden_layer_sizes=best_params['hidden_layer_sizes'],
                               activation=best_params['activation'],
                               solver=best_params['solver'],
                               alpha=best_params['alpha'],
                               learning_rate=best_params['learning_rate'],
                               random_state=42,
                               max_iter=5)




mlp_classifier.fit(X_train_features, y_train_indices)

train_predictions_indices = mlp_classifier.predict(X_train_features)
validation_predictions_indices = mlp_classifier.predict(X_val_features)

train_accuracy = accuracy_score(y_train_indices, train_predictions_indices)
validation_accuracy = accuracy_score(y_val_indices, validation_predictions_indices)

print("Training Accuracy:", train_accuracy)
print("Validation Accuracy:", validation_accuracy)

Training Accuracy: 0.8438642297650131
Validation Accuracy: 0.8517745302713987


In [10]:
train_cm = confusion_matrix(y_train_indices, train_predictions_indices)
validation_cm = confusion_matrix(y_val_indices, validation_predictions_indices)

train_class_accuracies = train_cm.diagonal() / train_cm.sum(axis=1)
validation_class_accuracies = validation_cm.diagonal() / validation_cm.sum(axis=1)

print("\nClass-wise Training Accuracies:")
for idx, acc in enumerate(train_class_accuracies):
    print(f"Accuracy for class {idx}: {acc:.2f}")

print("\nClass-wise Validation Accuracies:")
for idx, acc in enumerate(validation_class_accuracies):
    print(f"Accuracy for class {idx}: {acc:.2f}")


Class-wise Training Accuracies:
Accuracy for class 0: 0.37
Accuracy for class 1: 0.97
Accuracy for class 2: 0.00

Class-wise Validation Accuracies:
Accuracy for class 0: 0.31
Accuracy for class 1: 0.97
Accuracy for class 2: 0.00


In [12]:
print("Training Classification Report:")
print(classification_report(y_train_indices, train_predictions_indices))

print("\nValidation Classification Report:")
print(classification_report(y_val_indices, validation_predictions_indices))

Training Classification Report:
              precision    recall  f1-score   support

           0       0.71      0.37      0.49       314
           1       0.86      0.97      0.91      1539
           2       0.00      0.00      0.00        62

    accuracy                           0.84      1915
   macro avg       0.52      0.45      0.47      1915
weighted avg       0.80      0.84      0.81      1915


Validation Classification Report:
              precision    recall  f1-score   support

           0       0.51      0.31      0.38        62
           1       0.88      0.97      0.92       401
           2       0.00      0.00      0.00        16

    accuracy                           0.85       479
   macro avg       0.46      0.43      0.44       479
weighted avg       0.80      0.85      0.82       479



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classif

In [13]:
trained_model_folder = os.path.join(GOOGLE_DRIVE_PATH,'Models')

os.makedirs(trained_model_folder, exist_ok=True)

model_file_path = os.path.join(trained_model_folder, 'MLP_model.pkl')

joblib.dump(mlp_classifier, model_file_path)

print(f"Trained model saved at: {model_file_path}")

Trained model saved at: /content/drive/MyDrive/Colab Notebooks/Shreenika_CV_CW_Folder/Models/MLP_model.pkl


SVM+hog with Hyper parameter optimization

In [14]:
"""
Thi is SVM for face mask detection, it first converts the one-hot encoded labels
into indices and then implements grid search to determine best
hyperparameters using cross-validation. A new SVM classifier is trained using the
best parameters and extracted HOG features. Model is evaluated against
"""
y_train_indices = np.argmax(y_train, axis=1)
y_val_indices = np.argmax(y_val, axis=1)

param_grid = {
    'C': [0.1, 1, 10, 100],
    'kernel': ['linear', 'rbf', 'poly'],
    'gamma': ['scale', 'auto']
}

svm_classifier = SVC()

grid_search = GridSearchCV(svm_classifier, param_grid, cv=3, scoring='accuracy')
grid_search.fit(X_train_features, y_train_indices)

best_params = grid_search.best_params_

svm_classifier = SVC(C=best_params['C'], kernel=best_params['kernel'], gamma=best_params['gamma'])

svm_classifier.fit(X_train_features, y_train_indices)

train_predictions_indices = svm_classifier.predict(X_train_features)
validation_predictions_indices = svm_classifier.predict(X_val_features)

train_accuracy = accuracy_score(np.argmax(y_train, axis=1), train_predictions_indices)
validation_accuracy = accuracy_score(np.argmax(y_val, axis=1), validation_predictions_indices)

print("Training Accuracy:", train_accuracy)
print("Validation Accuracy:", validation_accuracy)

Training Accuracy: 1.0
Validation Accuracy: 0.9018789144050104


In [15]:
train_cm = confusion_matrix(y_train_indices, train_predictions_indices)
validation_cm = confusion_matrix(y_val_indices, validation_predictions_indices)

train_class_accuracies = train_cm.diagonal() / train_cm.sum(axis=1)
validation_class_accuracies = validation_cm.diagonal() / validation_cm.sum(axis=1)

print("\nClass-wise Training Accuracies:")
for idx, acc in enumerate(train_class_accuracies):
    print(f"Accuracy for class {idx}: {acc:.2f}")

print("\nClass-wise Validation Accuracies:")
for idx, acc in enumerate(validation_class_accuracies):
    print(f"Accuracy for class {idx}: {acc:.2f}")



Class-wise Training Accuracies:
Accuracy for class 0: 1.00
Accuracy for class 1: 1.00
Accuracy for class 2: 1.00

Class-wise Validation Accuracies:
Accuracy for class 0: 0.60
Accuracy for class 1: 0.98
Accuracy for class 2: 0.12


In [16]:
train_classification_report = classification_report(np.argmax(y_train, axis=1), train_predictions_indices)
validation_classification_report = classification_report(np.argmax(y_val, axis=1), validation_predictions_indices)

print("Training Classification Report:")
print(train_classification_report)

print("Validation Classification Report:")
print(validation_classification_report)

Training Classification Report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00       314
           1       1.00      1.00      1.00      1539
           2       1.00      1.00      1.00        62

    accuracy                           1.00      1915
   macro avg       1.00      1.00      1.00      1915
weighted avg       1.00      1.00      1.00      1915

Validation Classification Report:
              precision    recall  f1-score   support

           0       0.73      0.60      0.65        62
           1       0.92      0.98      0.95       401
           2       1.00      0.12      0.22        16

    accuracy                           0.90       479
   macro avg       0.88      0.57      0.61       479
weighted avg       0.90      0.90      0.89       479



In [17]:
model_file_path = os.path.join(trained_model_folder, 'SVM_model.pkl')

joblib.dump(svm_classifier, model_file_path)

print(f"Trained model saved at: {model_file_path}")

Trained model saved at: /content/drive/MyDrive/Colab Notebooks/Shreenika_CV_CW_Folder/Models/SVM_model.pkl


CNN with Data Augmentation

In [18]:
print("Number of images in training set:", len(X_train))
print("Number of labels in training set:", len(y_train))
print("Number of images in validation set:", len(X_val))
print("Number of labels in validation set:", len(y_val))


Number of images in training set: 1915
Number of labels in training set: 1915
Number of images in validation set: 479
Number of labels in validation set: 479


In [19]:
"""
This code trains a convolutional neural network model for face detection
using data augmentation. The CNN architecture is constructed
using keras sequential API and has multiple convolutional layers.
the output layer consists of three neurons with softmax activation for multiclass
classification problem. The model is compiled using adam optimizer
and then trained using augmented data and validated on validation data.
After training the model is evaluated on both original training
and validation data for accuracies.
"""

datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(128, 128, 3)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(3, activation='softmax')
])

model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

model_evaluation = model.fit(datagen.flow(X_train, y_train, batch_size=32),
                    steps_per_epoch=len(X_train) // 32, epochs=10,
                    validation_data=(X_val, y_val))

train_loss, train_acc = model.evaluate(train_images, train_labels)
print("Train Accuracy (Before Augmentation):", train_acc)

val_loss, val_acc = model.evaluate(X_val, y_val)
print("Validation Accuracy:", val_acc)

Epoch 1/10
59/59 [==============================] - 75s 1s/step - loss: 0.4212 - accuracy: 0.8603 - val_loss: 0.2505 - val_accuracy: 0.9290
Epoch 2/10
59/59 [==============================] - 79s 1s/step - loss: 0.3323 - accuracy: 0.9039 - val_loss: 0.2133 - val_accuracy: 0.9415
Epoch 3/10
59/59 [==============================] - 66s 1s/step - loss: 0.3042 - accuracy: 0.9076 - val_loss: 0.2070 - val_accuracy: 0.9332
Epoch 4/10
59/59 [==============================] - 60s 1s/step - loss: 0.2904 - accuracy: 0.9172 - val_loss: 0.2538 - val_accuracy: 0.9165
Epoch 5/10
59/59 [==============================] - 63s 1s/step - loss: 0.2990 - accuracy: 0.9108 - val_loss: 0.1759 - val_accuracy: 0.9478
Epoch 6/10
59/59 [==============================] - 64s 1s/step - loss: 0.2829 - accuracy: 0.9156 - val_loss: 0.1910 - val_accuracy: 0.9395
Epoch 7/10
59/59 [==============================] - 62s 1s/step - loss: 0.2825 - accuracy: 0.9182 - val_loss: 0.1885 - val_accuracy: 0.9374
Epoch 8/10
59/59 [==

In [20]:
train_predictions = model.predict(train_images)
train_predictions_classes = np.argmax(train_predictions, axis=1)
train_true_classes = np.argmax(train_labels, axis=1)

val_predictions = model.predict(X_val)
val_predictions_classes = np.argmax(val_predictions, axis=1)
val_true_classes = np.argmax(y_val, axis=1)

train_cm = confusion_matrix(train_true_classes, train_predictions_classes)
val_cm = confusion_matrix(val_true_classes, val_predictions_classes)

train_class_accuracies = train_cm.diagonal() / train_cm.sum(axis=1)
val_class_accuracies = val_cm.diagonal() / val_cm.sum(axis=1)

print("\nClass-wise Accuracies for Training Set:")
for idx, acc in enumerate(train_class_accuracies):
    print(f"Class {idx}: {acc:.2f}")

print("\nClass-wise Accuracies for Validation Set:")
for idx, acc in enumerate(val_class_accuracies):
    print(f"Class {idx}: {acc:.2f}")

15/15 [==============================] - 4s 227ms/step

Class-wise Accuracies for Training Set:
Class 0: 0.94
Class 1: 0.98
Class 2: 0.00

Class-wise Accuracies for Validation Set:
Class 0: 0.97
Class 1: 0.98
Class 2: 0.00


In [21]:
print("Classification Report for Training Data:")
print(classification_report(train_true_classes, train_predictions_classes))

print("Classification Report for Validation Data:")
print(classification_report(val_true_classes, val_predictions_classes))

Classification Report for Training Data:
              precision    recall  f1-score   support

           0       0.84      0.94      0.89       376
           1       0.96      0.98      0.97      1940
           2       0.00      0.00      0.00        78

    accuracy                           0.94      2394
   macro avg       0.60      0.64      0.62      2394
weighted avg       0.91      0.94      0.92      2394

Classification Report for Validation Data:
              precision    recall  f1-score   support

           0       0.85      0.97      0.90        62
           1       0.97      0.98      0.97       401
           2       0.00      0.00      0.00        16

    accuracy                           0.95       479
   macro avg       0.60      0.65      0.63       479
weighted avg       0.92      0.95      0.93       479



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classif

In [22]:
model_file_path = os.path.join(trained_model_folder, 'CNN_model.pkl')

joblib.dump(model, model_file_path)

print(f"Trained model saved at: {model_file_path}")

Trained model saved at: /content/drive/MyDrive/Colab Notebooks/Shreenika_CV_CW_Folder/Models/CNN_model.pkl
